# Phase2

# Connection

In [15]:

import pprint as pp
import requests

host = 'api.novasearch.org'
port = 443

user = 'user02' # Add your user name here.
password = 'marco.2025+' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

In [14]:
import urllib3
urllib3.disable_warnings()  # Suppress self-signed cert warnings

from requests.auth import HTTPBasicAuth
import requests
import pprint as pp

host = 'localhost'
port = 9200
user = 'admin'
password = 'MyStr0ng@Pass'
index_name = user

url = f"http://{host}:{port}"

try:
    res = requests.get(url, auth=HTTPBasicAuth(user, password), verify=False)
    pp.pprint(res.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)


Request failed: HTTPConnectionPool(host='localhost', port=9200): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024ED1764970>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


# Frame Extraction

In [21]:
import pandas as pd
from pathlib import Path
import json
import yt_dlp
import av # For frame extraction
import os
import time
import math # For ceiling function if needed for frame counts

# --- Provided load_captions function (remains the same) ---
def load_captions(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    result = []
    for vid_key, meta_info in data.items():
        if isinstance(meta_info, dict) and 'timestamps' in meta_info and 'sentences' in meta_info:
            for (start, end), sentence in zip(meta_info['timestamps'], meta_info['sentences']):
                result.append({
                    "video_id": vid_key,
                    "duration": meta_info.get("duration", 0),
                    "start": start,
                    "end": end,
                    "caption": sentence,
                    "video_url": f"https://www.youtube.com/watch?v={vid_key[2:]}" if vid_key.startswith("v_") else f"https://www.youtube.com/watch?v={vid_key}"
                })
        # else:
            # print(f"Warning: Skipping entry {vid_key} in load_captions due to unexpected structure or missing keys.")
    return result

# --- Setup ---
repo_root = Path().resolve()
json_path = repo_root / "captions" / "train.json"
OUTPUT_DIR = "extracted_frames"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- CONFIGURATION for frame extraction ---
FRAME_EXTRACTION_INTERVAL_SECONDS = 2.0

if not json_path.exists():
    print(f"Error: JSON file not found at {json_path}")
    exit()
# print(f"Loading captions from: {json_path}") # Reduced verbosity
data = load_captions(str(json_path))
if not data:
    print("No data loaded from captions file. Exiting.")
    exit()
df = pd.DataFrame(data)
if df.empty:
    print("DataFrame is empty after loading captions. Exiting.")
    exit()

# --- Filtering logic ---
contains_car = df[df['caption'].str.contains(r'\bcar\b', case=False, na=False)]
car_video_ids = set(contains_car['video_id'])
caption_counts = df['video_id'].value_counts()
videos_with_8plus = set(caption_counts[caption_counts > 8].index)
valid_video_ids = list(car_video_ids & videos_with_8plus)

# print(f"Found {len(valid_video_ids)} potentially valid video IDs based on caption criteria.") # Reduced verbosity

video_id_to_url_map = {}
if not df.empty and 'video_id' in df.columns and 'video_url' in df.columns:
    temp_df_for_urls = df[df['video_id'].isin(valid_video_ids)][['video_id', 'video_url']].drop_duplicates()
    video_id_to_url_map = pd.Series(temp_df_for_urls.video_url.values, index=temp_df_for_urls.video_id).to_dict()
else:
    print("DataFrame is missing 'video_id' or 'video_url' columns. Cannot proceed with URL mapping.")
    exit()

# --- Download videos and extract frames ---
TARGET_SUCCESSFUL_VIDEOS = 10
successfully_processed_count = 0
attempted_video_index = 0
successfully_downloaded_ids_and_framesinfo = []

print(f"\nStarting video processing (aiming for {TARGET_SUCCESSFUL_VIDEOS} successful videos)...")

while successfully_processed_count < TARGET_SUCCESSFUL_VIDEOS and attempted_video_index < len(valid_video_ids):
    video_id = valid_video_ids[attempted_video_index]
    attempted_video_index += 1

    url = video_id_to_url_map.get(video_id)
    if not url:
        print(f"Video ID {video_id}: Could not find URL. Skipping.")
        continue

    # print(f"\nAttempting to process video {attempted_video_index}/{len(valid_video_ids)} (target {successfully_processed_count+1}/{TARGET_SUCCESSFUL_VIDEOS}): {video_id}") # Reduced verbosity
    
    download_filename_template = f"{video_id}.mp4"
    temp_download_dir = os.path.join(OUTPUT_DIR, "temp_videos")
    os.makedirs(temp_download_dir, exist_ok=True)
    local_video_path_template = os.path.join(temp_download_dir, download_filename_template)
    
    actual_downloaded_video_path = None

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': local_video_path_template,
        'quiet': True, # Suppress yt-dlp console output
        'noplaylist': True,
        'socket_timeout': 30,
        'logtostderr': False, # Don't log to stderr when quiet
        'no_warnings': True, # Suppress warnings when quiet
    }

    try:
        # print(f"Processing video: {video_id}...") # Reduced verbosity, combined below
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            actual_downloaded_video_path = ydl.prepare_filename(info_dict)

            if not os.path.exists(actual_downloaded_video_path):
                 if os.path.exists(local_video_path_template):
                      actual_downloaded_video_path = local_video_path_template
                 else:
                      print(f"Video ID {video_id}: Download failed or file not found.")
                      continue
        
        # print(f"Downloaded {video_id} to {actual_downloaded_video_path}") # Reduced verbosity

        # print(f"Extracting frames for {video_id}...") # Reduced verbosity
        video_frames_output_dir = os.path.join(OUTPUT_DIR, video_id)
        os.makedirs(video_frames_output_dir, exist_ok=True)
        
        frames_extracted_count = 0
        with av.open(actual_downloaded_video_path) as container:
            stream = container.streams.video[0]
            fps = stream.average_rate
            if not fps or float(fps) == 0: fps = stream.guessed_rate
            if not fps or float(fps) == 0: fps = stream.r_frame_rate
            if not fps or float(fps) == 0:
                print(f"Video ID {video_id}: Could not determine FPS. Skipping frame extraction.")
                continue
            fps = float(fps)
            if fps <= 0:
                print(f"Video ID {video_id}: Invalid FPS ({fps}). Skipping frame extraction.")
                continue

            frames_to_skip_per_interval = int(round(fps * FRAME_EXTRACTION_INTERVAL_SECONDS))
            if frames_to_skip_per_interval <= 0: frames_to_skip_per_interval = 1
            
            # print(f"  Video FPS: {fps:.2f}, Extracting 1 frame every {frames_to_skip_per_interval} frames.") # Reduced verbosity

            frame_index_in_video = 0
            next_extraction_frame_index = 0

            for frame_obj in container.decode(stream):
                if frame_index_in_video >= next_extraction_frame_index:
                    timestamp_seconds = frame_obj.time
                    timestamp_str = f"{timestamp_seconds:.2f}".replace('.', '_')
                    out_frame_path = os.path.join(video_frames_output_dir, f"{video_id}_ts_{timestamp_str}_frame_{frames_extracted_count:04d}.jpg")
                    frame_obj.to_image().save(out_frame_path, quality=80)
                    frames_extracted_count += 1
                    next_extraction_frame_index += frames_to_skip_per_interval
                frame_index_in_video += 1
        
        if frames_extracted_count > 0:
            print(f"Video ID {video_id}: Success - {frames_extracted_count} frames extracted.")
            successfully_processed_count += 1
            successfully_downloaded_ids_and_framesinfo.append({'video_id': video_id, 'num_frames_extracted': frames_extracted_count, 'frames_path': video_frames_output_dir})
        else:
            print(f"Video ID {video_id}: Downloaded but no frames extracted (check FPS/length).")

    except yt_dlp.utils.DownloadError as e_dl:
        error_msg = str(e_dl).lower()
        # Simplified error reporting
        if "video unavailable" in error_msg: print(f"Video ID {video_id}: Failed - Video unavailable.")
        elif "private video" in error_msg: print(f"Video ID {video_id}: Failed - Private video.")
        elif "login required" in error_msg or "account action required" in error_msg: print(f"Video ID {video_id}: Failed - Login/account action required.")
        elif "http error 400" in error_msg: print(f"Video ID {video_id}: Failed - HTTP Error 400.")
        elif "http error 403" in error_msg: print(f"Video ID {video_id}: Failed - HTTP Error 403 (Forbidden).")
        elif "socket timeout" in error_msg: print(f"Video ID {video_id}: Failed - Socket timeout.")
        else: print(f"Video ID {video_id}: Failed - yt-dlp download error: {e_dl}")
    except Exception as e:
        print(f"Video ID {video_id}: Failed - Unexpected error: {e}")
        # import traceback # Optionally keep for debugging
        # traceback.print_exc()
    finally:
        if actual_downloaded_video_path and os.path.exists(actual_downloaded_video_path):
            try:
                # print(f"Cleaning up video file: {actual_downloaded_video_path}") # Reduced verbosity
                os.remove(actual_downloaded_video_path)
            except Exception as e_remove:
                print(f"Warning: Failed to remove video file {actual_downloaded_video_path}: {e_remove}")
    
    # time.sleep(0.5) # Keep if desired for rate limiting

print(f"\nFinished processing loop.")
print(f"{successfully_processed_count} of {attempted_video_index} attempted videos had frames successfully extracted.")
if successfully_processed_count < TARGET_SUCCESSFUL_VIDEOS and attempted_video_index == len(valid_video_ids):
    print(f"Target of {TARGET_SUCCESSFUL_VIDEOS} successful videos not reached. All {len(valid_video_ids)} potential videos processed.")
elif successfully_processed_count < TARGET_SUCCESSFUL_VIDEOS:
     print(f"Target of {TARGET_SUCCESSFUL_VIDEOS} successful videos not reached.")


if successfully_downloaded_ids_and_framesinfo:
    print("\nSummary of successfully processed videos:")
    total_frames_globally = 0
    for info in successfully_downloaded_ids_and_framesinfo:
        # Path is removed for less verbosity, can be added back if needed
        print(f"  - Video ID: {info['video_id']}, Frames Extracted: {info['num_frames_extracted']}")
        total_frames_globally += info['num_frames_extracted']
    print(f"\nTotal frames extracted across all successful videos: {total_frames_globally}")
else:
    print("\nNo videos were successfully processed or no frames were extracted.")



Starting video processing (aiming for 10 successful videos)...


ERROR: [youtube] MWdPh6J-YXM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Video ID v_MWdPh6J-YXM: Failed - Private video.
Video ID v_1LdbczjQPII: Success - 82 frames extracted.     
Video ID v_uICwWvS_AOo: Success - 96 frames extracted.     
Video ID v_Gms3Yt6RrV4: Success - 75 frames extracted.     


ERROR: [youtube] amCD-2TIKw0: Video unavailable


Video ID v_amCD-2TIKw0: Failed - Video unavailable.


ERROR: [youtube] jPLJAYnjsBw: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Video ID v_jPLJAYnjsBw: Failed - Private video.
Video ID v_Eilil6FZhK8: Success - 110 frames extracted.    
Video ID v_9DVsv84awMg: Success - 107 frames extracted.    
Video ID v_9hR1MHvXGv8: Success - 57 frames extracted.     


ERROR: [youtube] S5kuckj4Ud4: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Video ID v_S5kuckj4Ud4: Failed - Private video.
Video ID v_9V7cMp_w1_0: Success - 89 frames extracted.                   
Video ID v_06ofnvq2Hjs: Success - 102 frames extracted.    

Finished processing loop.
8 of 12 attempted videos had frames successfully extracted.
Target of 10 successful videos not reached. All 12 potential videos processed.

Summary of successfully processed videos:
  - Video ID: v_1LdbczjQPII, Frames Extracted: 82
  - Video ID: v_uICwWvS_AOo, Frames Extracted: 96
  - Video ID: v_Gms3Yt6RrV4, Frames Extracted: 75
  - Video ID: v_Eilil6FZhK8, Frames Extracted: 110
  - Video ID: v_9DVsv84awMg, Frames Extracted: 107
  - Video ID: v_9hR1MHvXGv8, Frames Extracted: 57
  - Video ID: v_9V7cMp_w1_0, Frames Extracted: 89
  - Video ID: v_06ofnvq2Hjs, Frames Extracted: 102

Total frames extracted across all successful videos: 718
